# Replicate project database from exported files

Use this notebook to set up a new brightway2 project on your computer, and load all the background and foreground databases, and project parameters, based on export files available online.

**Warning:** Only works if there are no loops between the foreground databases. (see Section Fix inadequate project otherwise)

**Warning:** If you import bw2packages, make sure you can trust the person who created the packages!

In [19]:
import brightway2 as bw2 # version v2.3
import eidl # https://github.com/haasad/EcoInventDownLoader
import glob # https://docs.python.org/3/library/glob.html#glob.glob 

from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group
import pandas as pd

def ExportProjectParameters(f='ProjectParamExport.xlsx'):
    params = { i:p.dict for i,p in enumerate(ProjectParameter.select() )}
    pd.DataFrame.from_dict(params).to_excel(f)

def LoadProjectParameters(f='ProjectParamExport.xlsx'):
    df = pd.read_excel(f, index_col=0)
    paramDict = df.to_dict(orient='dict')
    paramList = [v for k,v in paramDict.items()]
    bw2.parameters.new_project_parameters(paramList, overwrite=True)
    bw2.parameters.recalculate()

## Replicate

In [2]:
# Create a new project or set as current project
project_name = 'cs_UppsalaNewCopy'
if project_name not in bw2.projects:
    bw2.projects.create_project(project_name)
bw2.projects.set_current(project_name)

if "biosphere3" not in bw2.databases:
    bw2.bw2setup()

In [3]:
# import the ecoinvent database (requires license)
# and name it 'exactly' as stated in the documentation of the project your replicate
# slow process: min 15 minutes for 3 versions of ecoinvent!

ei_version = ['3.6', '3.6', '3.6']
ei_systemmodel = ['cut-off', 'apos', 'consequential']
ei_name = ['ei_cutoff_36', 'ei_apos_36', 'ei_csq_36']

username= '' # you ecoinvent username
password= '' # you ecoinvent password
for n, eiv in enumerate(ei_version):
    if ei_name[n] not in bw2.databases:
        eidl.get_ecoinvent(db_name=ei_name[n], # db_name: name to give imported database (string) default is downloaded filename
                           auto_write=True, # auto_write: automatically write database if no unlinked processes (boolean) default is False (i.e. prompt yes or no)
                           download_path=None, # download_path: path to download .7z file to (string) default is download to temporary directory (.7z file is deleted after import)
                           store_download=True, # store_download: store the .7z file for later reuse, default is True, only takes effect if no download_path is provided
                           username=username, # username: ecoinvent username (string)
                           password=password, # password: ecoivnent password (string)
                           version = eiv, # version: ecoinvent version (string), eg '3.5'
                           system_model=ei_systemmodel[n], # system_model: ecoinvent system model (string), one of {'cut-off', 'apos', 'consequential'}
                          )
    else:
        print("Database {} already in the project.".format(ei_name[n]))

Database ei_cutoff_36 already in the project.
Database ei_apos_36 already in the project.
Database ei_csq_36 already in the project.


In [4]:
# using BW2Packages - ONLY IF YOU TRUST THE FILES as a someone with bad intensions could hide executable code in the bw2package files
# import foreground databases and project parameters from files
where_are_the_files = 'C://Users//eazzi//AppData//Local//pylca//Brightway3//cs_UppsalaDuplicate.c641eb45289d00b26a72112a023a35f9\\export\\'

#import_order = [ 'background_system', 'material', 'material_tagged', 'pro_biochar', 
#                'use_1_tree-planting', 'use_2_green-roof', 'use_3_charcrete', 'use_4_filter', 'use_5_asphalt', 
#                'use_6_soil-private', 'use_7_benchmark', 'ranking']

import_order = ['AllForegroundInOne']

ordered_file_paths = []
for db in import_order:
    filesFound = glob.glob(where_are_the_files+db+'*.*', recursive=False) # returns a list
    if len(filesFound):
        ordered_file_paths.append(filesFound[0])
    elif len(filesFound)==0:
        raise Exception('Warning: no matching files for {} found in the folder. Verify import_order list and your folder content'.format(db))
    else:
        print('Warning: several matching files for {} found in the folder. Adding only first one'.format(db))
        ordered_file_paths.append(filesFound[0])

for fp in ordered_file_paths:
    bw2.BW2Package.import_file(fp)
    
# add project parameters
path_param = where_are_the_files+'ProjectParameters.xlsx'
df = pd.read_excel(path_param, index_col=0)
df = df.fillna('')
paramDict = df.to_dict(orient='dict')
paramList = [v for k,v in paramDict.items()]
bw2.parameters.new_project_parameters(paramList, overwrite=True)

for obj in Group.select().where(Group.fresh==False):
    data = [
        {'name':'FixParam',
        'database':'pro_biochar',
        'code':obj.name,
        'amount':'1',
        'formula':''}]
    bw2.parameters.new_activity_parameters(data=data, 
                                           group=obj.name, 
                                           overwrite=True)
bw2.parameters.recalculate()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/14/2021 16:40:51
  Finished: 09/14/2021 16:40:51
  Total time elapsed: 00:00:00
  CPU %: 77.80
  Memory %: 1.02


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/14/2021 16:40:52
  Finished: 09/14/2021 16:40:52
  Total time elapsed: 00:00:00
  CPU %: 100.30
  Memory %: 1.04


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


...

Title: Writing activities to SQLite3 database:
  Started: 09/14/2021 16:40:55
  Finished: 09/14/2021 16:40:55
  Total time elapsed: 00:00:00
  CPU %: 208.30
  Memory %: 1.05


Writing activities to SQLite3 database:
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/14/2021 16:40:55
  Finished: 09/14/2021 16:40:55
  Total time elapsed: 00:00:00
  CPU %: 75.60
  Memory %: 1.05


Writing activities to SQLite3 database:
0% [##############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/14/2021 16:40:56
  Finished: 09/14/2021 16:40:56
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.05


## Export

### via bw2packages

In [18]:
name_of_bw_project = "cs_UppsalaDuplicate"
bw2.projects.set_current(name_of_bw_project)
print("Current project is {}".format(bw2.projects.current))

# THE ORDER IN THIS LIST IS IMPORTANT (e.g. ranking at the end, because depends on all other uses)
objects_to_copy = [
'background_system',
'material',
'material_tagged',
'pro_biochar',
'use_1_tree-planting',
'use_2_green-roof',
'use_3_charcrete',
'use_4_filter',
'use_5_asphalt',
'use_6_soil-private',
'use_7_benchmark',
'ranking']

bw2obj = []
for o in objects_to_copy:
    print(o)
    #bw2.Database(o).process()
    bw2obj.append(bw2.Database(o))

pack_path = bw2.BW2Package.export_objs(objs=bw2obj, filename='AllForegroundInOne', folder='export')
path_param = pack_path.rsplit('\\', 1)[0]+'\\ProjectParameters.xlsx'
ExportProjectParameters(f=path_param)
print('Exported files are in:')
print(pack_path.rsplit('\\', 1)[0])

Current project is cs_UppsalaDuplicate
background_system
material
material_tagged
pro_biochar
use_1_tree-planting
use_2_green-roof
use_3_charcrete
use_4_filter
use_5_asphalt
use_6_soil-private
use_7_benchmark
ranking
Exported files are in:
C:\Users\eazzi\AppData\Local\pylca\Brightway3\cs_UppsalaDuplicate.c641eb45289d00b26a72112a023a35f9\export


### Via Excel files or JSON-LD (to do)

In [ ]:
name_of_bw_project = "cs_UppsalaDuplicate"
bw2.projects.set_current(name_of_bw_project)
print("Current project is {}".format(bw2.projects.current))

# THE ORDER IN THIS LIST IS IMPORTANT (e.g. ranking at the end, because depends on all other uses)
objects_to_copy = [
'background_system',
'material',
'material_tagged',
'pro_biochar',
'use_1_tree-planting',
'use_2_green-roof',
'use_3_charcrete',
'use_4_filter',
'use_5_asphalt',
'use_6_soil-private',
'use_7_benchmark',
'ranking']

In [25]:
# https://2.docs.brightway.dev/technical/bw2io.html?highlight=export#excel
import bw2io
bw2io.export.excel.write_lci_excel?
# nested data would not be exported! 

## Fix inadequate project setup

In case the project you want to copy has several interlinked databases, or if you want to merge some databases, or if you want to rename + relink a database, before you export your files for sharing: explore & adapt the code below.

In [ ]:
## Merge 2 foregroundDatabases in one & relink them internally?
name_of_bw_project = "cs_UppsalaDuplicate"
bw2.projects.set_current(name_of_bw_project)
print("Current project is {}".format(bw2.projects.current))

merge_in = 'background_system'
to_merge = ['biomass', 'background-system', 'biomass_test']

if merge_in not in bw2.databases:
    bw2.Database(merge_in).register()

# copy all activities in a merged database
counter = 0
for db in to_merge:
    for a in bw2.Database(db):
        if a not in bw2.Database(merge_in):
            a.copy(code=a['code'], database=merge_in) # copies everything, including tags, comments, etc
            counter+=1
print("Copied {} activities to the new database '{}' ".format(counter, merge_in) )

from bw2io.utils import DEFAULT_FIELDS, activity_hash

# modify all the exchanges in the merged database, so they link to internal activities
for old_db in to_merge:
    print("Relinking for db {}".format(old_db))
    if old_db == merge_in:
        print("No point relinking to same database.")
        break
        
    db = bw2.Database(merge_in)
    other = bw2.Database(old_db)
    assert db.backend == "sqlite", "Relinking only allowed for SQLITE backends"
    assert other.backend == "sqlite", "Relinking only allowed for SQLITE backends"

    # find dupplicates & candidates between old & new database
    duplicates, candidates = {}, {}
    altered = 0
    for ds in db: # for activities in new database
        # key = activity_hash(ds, DEFAULT_FIELDS) # creates a hash based on default field of activity # to avoid
        key = ds.key[1] # because I have simply copied them, and they may have _copy1 extensions
        if key in candidates: # candidates is empty at first
            duplicates.setdefault(key, []).append(ds)
        else:
            candidates[key] = ds.key # key is just a hash, ds.key is (code, hash)
            
    # traverse all the activities and their technosphere/biosphere exchanges, for changing the exchanges
    for i, exc in enumerate(
                        exc for act in db for exc in act.exchanges()
                        if exc.get("type") in {"biosphere", "technosphere"} and exc.input[0] == old_db
                ):
                    # Use the input activity to generate the hash.
                    key = exc.input.key[1] #activity_hash(exc.input, DEFAULT_FIELDS)
                    if key in duplicates:
                        raise StrategyError(format_nonunique_key_error(exc.input, DEFAULT_FIELDS, duplicates[key]))
                    elif key in candidates:
                        exc["input"] = candidates[key]
                        altered += 1
                    exc.save()
    print('-- {} exchanges were modified'.format(altered))
    
    db.process()
    print(
        "Relinked database '{}', {} exchange inputs changed from '{}' to '{}'.".format(
            db.name, altered, other.name, db.name
        )
    )

other_fgdb = [
'material',
'material_tagged',
'pro_biochar',
'ranking',
'use_1_tree-planting',
'use_2_green-roof',
'use_3_charcrete',
'use_4_filter',
'use_5_asphalt',
'use_6_soil private',
'use_7_benchmark']

for fg_db in other_fgdb:
    fg_db = bw2.Database(fg_db) # the db to relink
    new_db = bw2.Database(merge_in) # new db
    
    for old_db in to_merge:
        old_db = bw2.Database(old_db) # old db

        # find dupplicates & candidates between old & new database
        duplicates, candidates = {}, {}
        altered = 0
        for ds in new_db: # for activities in new database
            # key = activity_hash(ds, DEFAULT_FIELDS) # creates a hash based on default field of activity # to avoid
            key = ds.key[1] # because I have simply copied them, and they may have _copy1 extensions
            if key in candidates: # candidates is empty at first
                duplicates.setdefault(key, []).append(ds)
            else:
                candidates[key] = ds.key # key is just a hash, ds.key is (code, hash)

        # traverse all the activities and their technosphere/biosphere exchanges, for changing the exchanges
        for i, exc in enumerate(
                            exc for act in fg_db for exc in act.exchanges()
                            if exc.get("type") in {"biosphere", "technosphere"} and exc.input[0] == old_db.name
                    ):
                        # Use the input activity to generate the hash.
                        key = exc.input.key[1] 
                        if key in duplicates:
                            raise StrategyError(format_nonunique_key_error(exc.input, DEFAULT_FIELDS, duplicates[key]))
                        elif key in candidates:
                            exc["input"] = candidates[key]
                            altered += 1
                        exc.save()

        fg_db.process()
        print(
            "Relinked database '{}', {} exchange inputs changed from '{}' to '{}'.".format(
                fg_db.name, altered, old_db.name, new_db.name
            )
        )

In [ ]:
# Trying to rename a database, to remove a ' ' space in its name
# Trick: don't use rename, gonna mess up the things
# Instead: merge one into one other + relink all others, then delete old one

name_of_bw_project = "cs_UppsalaDuplicate"
bw2.projects.set_current(name_of_bw_project)
print("Current project is {}".format(bw2.projects.current))


merge_in = 'use_6_soil-private'
to_merge = ['use_6_soil private']

if merge_in not in bw2.databases:
    bw2.Database(merge_in).register()

# copy all activities in a merged database
counter = 0
for db in to_merge:
    for a in bw2.Database(db):
        if a not in bw2.Database(merge_in):
            a.copy(code=a['code'], database=merge_in) # copies everything, including tags, comments, etc
            counter+=1
print("Copied {} activities to the new database '{}' ".format(counter, merge_in) )

# modify all the exchanges in the merged database, so they link to internal activities
for old_db in to_merge:
    print("Relinking for db {}".format(old_db))
    if old_db == merge_in:
        print("No point relinking to same database.")
        break
        
    db = bw2.Database(merge_in)
    other = bw2.Database(old_db)
    assert db.backend == "sqlite", "Relinking only allowed for SQLITE backends"
    assert other.backend == "sqlite", "Relinking only allowed for SQLITE backends"

    # find dupplicates & candidates between old & new database
    duplicates, candidates = {}, {}
    altered = 0
    for ds in db: # for activities in new database
        # key = activity_hash(ds, DEFAULT_FIELDS) # creates a hash based on default field of activity # to avoid
        key = ds.key[1] # because I have simply copied them, and they may have _copy1 extensions
        if key in candidates: # candidates is empty at first
            duplicates.setdefault(key, []).append(ds)
        else:
            candidates[key] = ds.key # key is just a hash, ds.key is (code, hash)
            
    # traverse all the activities and their technosphere/biosphere exchanges, for changing the exchanges
    for i, exc in enumerate(
                        exc for act in db for exc in act.exchanges()
                        if exc.get("type") in {"biosphere", "technosphere"} and exc.input[0] == old_db
                ):
                    # Use the input activity to generate the hash.
                    key = exc.input.key[1] #activity_hash(exc.input, DEFAULT_FIELDS)
                    if key in duplicates:
                        raise StrategyError(format_nonunique_key_error(exc.input, DEFAULT_FIELDS, duplicates[key]))
                    elif key in candidates:
                        exc["input"] = candidates[key]
                        altered += 1
                    exc.save()
    print('-- {} exchanges were modified'.format(altered))
    
    db.process()
    print(
        "Relinked database '{}', {} exchange inputs changed from '{}' to '{}'.".format(
            db.name, altered, other.name, db.name
        )
    )

other_fgdb = [
'material',
'material_tagged',
'pro_biochar',
'ranking',
'use_1_tree-planting',
'use_2_green-roof',
'use_3_charcrete',
'use_4_filter',
'use_5_asphalt',
'use_6_soil private',
'use_7_benchmark']

for fg_db in other_fgdb:
    fg_db = bw2.Database(fg_db) # the db to relink
    new_db = bw2.Database(merge_in) # new db
    
    for old_db in to_merge:
        old_db = bw2.Database(old_db) # old db

        # find dupplicates & candidates between old & new database
        duplicates, candidates = {}, {}
        altered = 0
        for ds in new_db: # for activities in new database
            # key = activity_hash(ds, DEFAULT_FIELDS) # creates a hash based on default field of activity # to avoid
            key = ds.key[1] # because I have simply copied them, and they may have _copy1 extensions
            if key in candidates: # candidates is empty at first
                duplicates.setdefault(key, []).append(ds)
            else:
                candidates[key] = ds.key # key is just a hash, ds.key is (code, hash)

        # traverse all the activities and their technosphere/biosphere exchanges, for changing the exchanges
        for i, exc in enumerate(
                            exc for act in fg_db for exc in act.exchanges()
                            if exc.get("type") in {"biosphere", "technosphere"} and exc.input[0] == old_db.name
                    ):
                        # Use the input activity to generate the hash.
                        key = exc.input.key[1] 
                        if key in duplicates:
                            raise StrategyError(format_nonunique_key_error(exc.input, DEFAULT_FIELDS, duplicates[key]))
                        elif key in candidates:
                            exc["input"] = candidates[key]
                            altered += 1
                        exc.save()

        fg_db.process()
        print(
            "Relinked database '{}', {} exchange inputs changed from '{}' to '{}'.".format(
                fg_db.name, altered, old_db.name, new_db.name
            )
        )